# Import libraries

In [1]:
%matplotlib inline
import scipy.io # To use the '.mat' files
import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.signal import butter, filtfilt
import cv2
from matplotlib.colors import Normalize
from matplotlib import cm
from PIL import Image

In [2]:
pip install librosa matplotlib numpy

# Mount google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Automation code for spectrogram image generation using virtual sampling technique

In [4]:
def calculate_hop_length(segment_length, overlap_percent):
    return int(segment_length * (1 - overlap_percent / 100))

def process_csv(file_path, output_directory):
    data = pd.read_csv(file_path)
    new_data = np.array(data.iloc[:, 3])
    fs_original = 50000  # Original sample rate
    # Step 1: Resample to make a 10-second signal
    fs_new = 25000  # New sample rate to stretch the signal to 10 seconds
    data_new_streched = librosa.resample(new_data, orig_sr=fs_original, target_sr=fs_new)
    t_original = np.linspace(0, 5, len(new_data))
    # Time vectors for plotting
    t_stretched = np.linspace(0, 10, len(data_new_streched))  # 10 seconds for the stretched signal
    # Step 2: Extract the first 2 seconds from the stretched signal
    duration_extract = 2 # Duration to extract in seconds
    signal_extracted = data_new_streched[:int(fs_new * duration_extract)]
    signal = signal_extracted
    fs=25000
    n_fft = 5000
    segment_length = 5000
    hop_length = calculate_hop_length(segment_length, 96)
    Fmax=1120
    # Determine which frequency bins to keep
    freqs = np.fft.rfftfreq(n_fft, 1/fs)
    max_index = np.where(freqs <= Fmax)[0][-1] + 1  # Last index where the frequency is <= 5000 Hz
    # Initialize spectrogram array with the correct number of rows
    spectrogram = np.zeros((max_index, 0))
    # Window function
    window = np.hanning(n_fft)
    # Process segments
    current_position = 0
    while current_position + n_fft <= len(signal):
        end = current_position + n_fft
        segment = signal[current_position:end] * window
        fft_result = np.fft.rfft(segment, n=n_fft)
        fft_result = fft_result[:max_index]  # Keep only the components up to 5000 Hz
        frequencies = np.fft.rfftfreq(n_fft, 1/fs_new)
        spectrogram = np.hstack((spectrogram, np.abs(fft_result).reshape(-1, 1)))
        NN=librosa.power_to_db(spectrogram, ref=np.max)
        current_position += hop_length
    # print("Frequency resolution (Hz):", frequencies[1]- frequencies[0])
    # Resize the spectrogram to 224x224
    spectrogram_resized = cv2.resize(NN,(224, 224), interpolation=cv2.INTER_CUBIC)
    normalized_spectrogram = (spectrogram_resized - np.min(spectrogram_resized)) / (np.max(spectrogram_resized) - np.min(spectrogram_resized))
    viridis = cm.get_cmap('viridis', 256)
    stft_colored = viridis(normalized_spectrogram)
    rgb_image_no_alpha = stft_colored[:, :, :3]
    # stft_colored_new_bgr = stft_colored_new[:, :, [2, 1, 0]]  # Reordering to BGR
    # output_image_path="/content/1.png"
    # cv2.imwrite(output_image_path, stft_colored_new_bgr)
    # Save the RGB image
    image = Image.fromarray((rgb_image_no_alpha * 255).astype(np.uint8))
    flipped_image = image.transpose(Image.FLIP_TOP_BOTTOM)
    # flipped_image.save('/content/resized_rgb_image.png')
    # Save the image
    image_new =  flipped_image
    filename = os.path.basename(file_path)
    output_path = os.path.join(output_directory, f"{os.path.splitext(filename)[0]}.png")
    image_new.save(output_path)
    # print(f"Saved {output_path}")
    # Parameters
    frequency_resolution = 5  # Hz per bin
    time_resolution = 0.008   # seconds per bin
    # Generate axes for plotting
    frequency_axis = np.arange(spectrogram.shape[0]) * frequency_resolution
    time_axis = np.arange(spectrogram.shape[1]) * time_resolution

    # Assuming 'spectrogram' and its related 'time_axis' and 'frequency_axis' are defined
    fig, axs = plt.subplots(1, 2, figsize=(20, 6))  # 1 row, 2 columns
    # Plot on the second axis
    img = axs[0].imshow(spectrogram, aspect='auto', origin='lower', extent=[time_axis[0], time_axis[-1], frequency_axis[0], frequency_axis[-1]])
    axs[0].set_title('Spectrogram with time and frequency')
    axs[0].set_xlabel('Time (seconds)')
    axs[0].set_ylabel('Frequency (Hz)')
    fig.colorbar(img, ax=axs[0], label='Magnitude')
    # Plot on the second axis
    img = axs[1].imshow(image, aspect='auto', origin='lower')
    axs[1].set_title('Spectrogram with time and frequency')
    axs[1].set_xlabel('Time bins')
    axs[1].set_ylabel('Frequency bins)')
    fig.colorbar(img, ax=axs[1], label='Magnitude')


In [5]:
# Directory containing the CSV files
directory = '/content/drive/MyDrive/New_MF_data/New_Class/BCO'
output_directory = '/content/drive/MyDrive/New_MF_data/New_Class/1HZ_Resolution_1000Hz_resized_spectrogram/BCO'

# Ensure output directory exists
os.makedirs(output_directory, exist_ok=True)

# Iterate over all CSV files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        process_csv(file_path, output_directory)

print("All files processed.")